# Source Mapping

Learn how to use bidirectional text ↔ structure mapping with `RenderedPrompt`.

In [1]:
from t_prompts import prompt

## Rendering and Source Maps

The `render()` method returns a `RenderedPrompt` with the final text and a source map that tracks which parts of the text came from which interpolations.

In [2]:
name = "Alice"
age = "30"
p = prompt(t"Name: {name:n}, Age: {age:a}")

rendered = p.render()

print(f"Text: {rendered.text}")
print(f"\nSource map ({len(rendered.source_map)} spans):")
for span in rendered.source_map:
    print(f"  [{span.start}:{span.end}] key='{span.key}' -> \"{rendered.text[span.start:span.end]}\"")

Text: Name: Alice, Age: 30

Source map (2 spans):
  [6:11] key='n' -> "Alice"
  [18:20] key='a' -> "30"


## Finding Source by Position

Use `get_span_at(pos)` to find which interpolation produced a specific position in the rendered text.

In [3]:
# Position 6 is in "Alice"
span = rendered.get_span_at(6)
print(f"Position 6:")
print(f"  Key: {span.key}")
print(f"  Span: [{span.start}:{span.end}]")
print(f"  Text: \"{rendered.text[span.start:span.end]}\"")

# Position 18 is in "30"
span = rendered.get_span_at(18)
print(f"\nPosition 18:")
print(f"  Key: {span.key}")
print(f"  Span: [{span.start}:{span.end}]")
print(f"  Text: \"{rendered.text[span.start:span.end]}\"")

Position 6:
  Key: n
  Span: [6:11]
  Text: "Alice"

Position 18:
  Key: a
  Span: [18:20]
  Text: "30"


## Finding Position by Key

Use `get_span_for_key(key)` to find where a specific key was rendered in the text.

In [4]:
# Find where 'n' (name) was rendered
span_n = rendered.get_span_for_key('n')
print(f"Key 'n':")
print(f"  Span: [{span_n.start}:{span_n.end}]")
print(f"  Text: \"{rendered.text[span_n.start:span_n.end]}\"")

# Find where 'a' (age) was rendered
span_a = rendered.get_span_for_key('a')
print(f"\nKey 'a':")
print(f"  Span: [{span_a.start}:{span_a.end}]")
print(f"  Text: \"{rendered.text[span_a.start:span_a.end]}\"")

Key 'n':
  Span: [6:11]
  Text: "Alice"

Key 'a':
  Span: [18:20]
  Text: "30"


## Source Mapping for Nested Prompts

Source maps work recursively for nested prompts. Each span includes a `path` tuple showing the nesting hierarchy.

In [5]:
greeting = "Hello"
name = "Alice"  # Re-use from earlier example
inner = prompt(t"{greeting:g}, {name:n}!")
outer = prompt(t"Message: {inner:msg}")

rendered_nested = outer.render()

print(f"Text: {rendered_nested.text}")
print(f"\nSource map with paths:")
for span in rendered_nested.source_map:
    print(f"  path={span.path} key='{span.key}' -> \"{rendered_nested.text[span.start:span.end]}\"")

Text: Message: Hello, Alice!

Source map with paths:
  path=('msg',) key='g' -> "Hello"
  path=('msg',) key='n' -> "Alice"


## Accessing the Original Prompt

The `RenderedPrompt` keeps a reference to the original `StructuredPrompt`.

In [6]:
# Access the source prompt
assert rendered_nested.source_prompt is outer

# Use it to get metadata
span = rendered_nested.get_span_for_key('msg')
node = rendered_nested.source_prompt['msg']
print(f"Key 'msg':")
print(f"  Expression: {node.expression}")
print(f"  Type: {type(node.value).__name__}")

Key 'msg':
  Expression: inner
  Type: StructuredPrompt


## Use Case: Highlighting in a UI

Source maps enable building UIs that highlight which parts of a rendered prompt came from which variables.

In [7]:
def highlight_key(rendered, key, marker='**'):
    """Highlight a specific key in the rendered text."""
    span = rendered.get_span_for_key(key)
    text = rendered.text
    return text[:span.start] + marker + text[span.start:span.end] + marker + text[span.end:]

city = "Paris"
country = "France"
p = prompt(t"{city:c} is the capital of {country:co}.")
r = p.render()

print("Original:")
print(r.text)
print("\nHighlight 'c':")
print(highlight_key(r, 'c'))
print("\nHighlight 'co':")
print(highlight_key(r, 'co'))

Original:
Paris is the capital of France.

Highlight 'c':
**Paris** is the capital of France.

Highlight 'co':
Paris is the capital of **France**.


## Use Case: Error Attribution

If an LLM returns an error pointing to a character position, you can trace it back to the source variable.

In [8]:
def trace_error_to_source(rendered, error_pos):
    """Trace an error position back to its source variable."""
    try:
        span = rendered.get_span_at(error_pos)
        if span is None:
            # Position is in a literal string, not an interpolation
            return None
        node = rendered.source_prompt[span.key]
        return {
            'key': span.key,
            'expression': node.expression,
            'value': node.value,
            'text_span': rendered.text[span.start:span.end]
        }
    except (KeyError, AttributeError):
        return None

# Simulate an error at position 18 (in "30")
error_info = trace_error_to_source(rendered, 18)
if error_info:
    print(f"Error at position 18 traced to:")
    print(f"  Key: {error_info['key']}")
    print(f"  Expression: {error_info['expression']}")
    print(f"  Value: {error_info['value']}")
    print(f"  Text: \"{error_info['text_span']}\"")
else:
    print("Position is not in an interpolation.")

Error at position 18 traced to:
  Key: a
  Expression: age
  Value: 30
  Text: "30"
